In [1]:
from os import listdir, path
import numpy as np
import scipy, cv2, os, sys, argparse, audio
import json, subprocess, random, string
from tqdm import tqdm
from glob import glob
import torch, face_detection
from models import Wav2Lip
import platform
import time
import sys

sys.argv = sys.argv[:1]
sys.argv += ['--checkpoint_path','checkpoints/wav2lip_gan.pth','--face','data/1min.mp4','--audio','data/long.wav']

parser = argparse.ArgumentParser(description='Inference code to lip-sync videos in the wild using Wav2Lip models')

parser.add_argument('--checkpoint_path', type=str, 
                    help='Name of saved checkpoint to load weights from', required=True)

parser.add_argument('--face', type=str, 
                    help='Filepath of video/image that contains faces to use', required=True)
parser.add_argument('--audio', type=str, 
                    help='Filepath of video/audio file to use as raw audio source', required=True)
parser.add_argument('--outfile', type=str, help='Video path to save result. See default for an e.g.', 
                                default='results/result_voice.mp4')

parser.add_argument('--static', type=bool, 
                    help='If True, then use only first video frame for inference', default=False)
parser.add_argument('--fps', type=float, help='Can be specified only if input is a static image (default: 25)', 
                    default=25., required=False)

parser.add_argument('--pads', nargs='+', type=int, default=[0, 10, 0, 0], 
                    help='Padding (top, bottom, left, right). Please adjust to include chin at least')

parser.add_argument('--face_det_batch_size', type=int, 
                    help='Batch size for face detection', default=8)
parser.add_argument('--wav2lip_batch_size', type=int, help='Batch size for Wav2Lip model(s)', default=512)

parser.add_argument('--resize_factor', default=1, type=int, 
                    help='Reduce the resolution by this factor. Sometimes, best results are obtained at 480p or 720p')

parser.add_argument('--crop', nargs='+', type=int, default=[0, -1, 0, -1], 
                    help='Crop video to a smaller region (top, bottom, left, right). Applied after resize_factor and rotate arg. ' 
                    'Useful if multiple face present. -1 implies the value will be auto-inferred based on height, width')

parser.add_argument('--box', nargs='+', type=int, default=[-1, -1, -1, -1], 
                    help='Specify a constant bounding box for the face. Use only as a last resort if the face is not detected.'
                    'Also, might work only if the face is not moving around much. Syntax: (top, bottom, left, right).')

parser.add_argument('--rotate', default=False, action='store_true',
                    help='Sometimes videos taken from a phone can be flipped 90deg. If true, will flip video right by 90deg.'
                    'Use if you get a flipped result, despite feeding a normal looking video')

parser.add_argument('--nosmooth', default=False, action='store_true',
                    help='Prevent smoothing face detections over a short temporal window')

args = parser.parse_args()
args.img_size = 96

if os.path.isfile(args.face) and args.face.split('.')[1] in ['jpg', 'png', 'jpeg']:
    args.static = True

def get_smoothened_boxes(boxes, T):
    for i in range(len(boxes)):
        if i + T > len(boxes):
            window = boxes[len(boxes) - T:]
        else:
            window = boxes[i : i + T]
        boxes[i] = np.mean(window, axis=0)
    return boxes

def face_detect(images):
    detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D, 
                                            flip_input=False, device=device)

    batch_size = args.face_det_batch_size
    
    while 1:
        predictions = []
        try:
            for i in tqdm(range(0, len(images), batch_size)):
                predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
        except RuntimeError:
            if batch_size == 1: 
                raise RuntimeError('Image too big to run face detection on GPU. Please use the --resize_factor argument')
            batch_size //= 2
            print('Recovering from OOM error; New batch size: {}'.format(batch_size))
            continue
        break

    results = []
    pady1, pady2, padx1, padx2 = args.pads
    for rect, image in zip(predictions, images):
        if rect is None:
            cv2.imwrite('temp/faulty_frame.jpg', image) # check this frame where the face was not detected.
            raise ValueError('Face not detected! Ensure the video contains a face in all the frames.')

        y1 = max(0, rect[1] - pady1)
        y2 = min(image.shape[0], rect[3] + pady2)
        x1 = max(0, rect[0] - padx1)
        x2 = min(image.shape[1], rect[2] + padx2)
        
        results.append([x1, y1, x2, y2])

    boxes = np.array(results)
    if not args.nosmooth: boxes = get_smoothened_boxes(boxes, T=5)
    results = [[image[y1: y2, x1:x2], (y1, y2, x1, x2)] for image, (x1, y1, x2, y2) in zip(images, boxes)]

    del detector
    return results

def datagen(frames, mels, face_results, batch_size = 128):
    img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []
    face_det_results = face_results
    
    for i, m in enumerate(mels):
        idx = 0 if args.static else i%len(frames)
        frame_to_save = frames[idx].copy()
        face, coords = face_det_results[idx].copy()
        
        face = cv2.resize(face, (args.img_size, args.img_size))
        
        img_batch.append(face)
        mel_batch.append(m)
        frame_batch.append(frame_to_save)
        coords_batch.append(coords)
        
        if len(img_batch) >= batch_size:
            img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

            img_masked = img_batch.copy()
            img_masked[:, args.img_size//2:] = 0

            img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
            mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])
            yield img_batch, mel_batch, frame_batch, coords_batch
            img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if len(img_batch) > 0:
        img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

        img_masked = img_batch.copy()
        img_masked[:, args.img_size//2:] = 0

        img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
        mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

        yield img_batch, mel_batch, frame_batch, coords_batch
    
    
def datagen_parallel(data_chunk):
    frames, mels, face_results = data_chunk
    
    img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []
    face_det_results = face_results
    
    for i, m in enumerate(mels):
        idx = 0 if args.static else i%len(frames)
        frame_to_save = frames[idx].copy()
        face, coords = face_det_results[idx].copy()

        face = cv2.resize(face, (args.img_size, args.img_size))
            
        img_batch.append(face)
        mel_batch.append(m)
        frame_batch.append(frame_to_save)
        coords_batch.append(coords)

    img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)
    img_masked = img_batch.copy()
    img_masked[:, args.img_size//2:] = 0
    img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
    mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

    return img_batch, mel_batch, frame_batch, coords_batch

mel_step_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} for inference.'.format(device))

def _load(checkpoint_path):
    if device == 'cuda':
        checkpoint = torch.load(checkpoint_path)
    else:
        checkpoint = torch.load(checkpoint_path,
                                map_location=lambda storage, loc: storage)
    return checkpoint

def load_model(path):
    model = Wav2Lip()
    print("Load checkpoint from: {}".format(path))
    checkpoint = _load(path)
    s = checkpoint["state_dict"]
    new_s = {}
    for k, v in s.items():
        new_s[k.replace('module.', '')] = v
    model.load_state_dict(new_s)

    model = model.to(device)
    return model.eval()

/home/sa47/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sa47/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sa47/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sa47/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/

Using cuda for inference.


In [2]:
if not os.path.isfile(args.face):
    raise ValueError('--face argument must be a valid path to video/image file')

elif args.face.split('.')[1] in ['jpg', 'png', 'jpeg']:
    full_frames = [cv2.imread(args.face)]
    fps = args.fps

else:
    video_stream = cv2.VideoCapture(args.face)
    fps = video_stream.get(cv2.CAP_PROP_FPS)

    print('Reading video frames...')

    full_frames = []
    while 1:
        still_reading, frame = video_stream.read()
        if not still_reading:
            video_stream.release()
            break
        if args.resize_factor > 1:
            frame = cv2.resize(frame, (frame.shape[1]//args.resize_factor, frame.shape[0]//args.resize_factor))

        if args.rotate:
            frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)

        y1, y2, x1, x2 = args.crop
        if x2 == -1: x2 = frame.shape[1]
        if y2 == -1: y2 = frame.shape[0]

        frame = frame[y1:y2, x1:x2]

        full_frames.append(frame)

print ("Number of frames available for inference: "+str(len(full_frames)))

if args.box[0] == -1:
    if not args.static:
        face_det_results = face_detect(full_frames.copy()) # BGR2RGB for CNN face detection
    else:
        face_det_results = face_detect([full_frames.copy()[0]])
else:
    y1, y2, x1, x2 = args.box
    face_det_results = [[f[y1: y2, x1:x2], (y1, y2, x1, x2)] for f in frames]

Reading video frames...
Number of frames available for inference: 2158


  0%|          | 0/540 [00:00<?, ?it/s]

Recovering from OOM error; New batch size: 4


100%|██████████| 540/540 [02:36<00:00,  3.45it/s]


In [3]:
args.checkpoint_path = 'checkpoints/wav2lip_gan.pth'
model = load_model(args.checkpoint_path)

Load checkpoint from: checkpoints/wav2lip_gan.pth


In [4]:
#args.audio = 'data/input_audio.mp3'
args.audio = 'data/long.wav'

In [5]:
len(full_frames)

2158

In [6]:
start = time.time()
# # if not args.audio.endswith('.wav'):
# print('Extracting raw audio...')
# command = 'ffmpeg -y -i {} -strict -2 {}'.format(args.audio, 'temp/temp.wav')
# subprocess.call(command, shell=True)
# print(time.time() - start)
# start = time.time()
# args.audio = 'temp/temp.wav'
wav = audio.load_wav(args.audio, 16000)
mel = audio.melspectrogram(wav)
print(time.time() - start)
start = time.time()

mel_chunks = []
mel_idx_multiplier = 80./fps 
i = 0
while 1:
    start_idx = int(i * mel_idx_multiplier)
    if start_idx + mel_step_size > len(mel[0]):
        mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
        break
    mel_chunks.append(mel[:, start_idx : start_idx + mel_step_size])
    i += 1

print(time.time() - start)
print("Length of mel chunks: {}".format(len(mel_chunks)))

MAX_FRAME = min(len(mel_chunks), len(full_frames))

start = time.time()
select_frames = full_frames[:MAX_FRAME].copy()
select_face_results = face_det_results[:MAX_FRAME].copy()

mel_chunks = mel_chunks[:MAX_FRAME]

0.12066793441772461
0.006732940673828125
Length of mel chunks: 2395


In [7]:
len(mel_chunks)

2158

In [8]:
start = time.time()

gen = list(datagen(select_frames, mel_chunks, select_face_results, 32))

print("Gen:", time.time() - start)

final_frames = []

start = time.time()
for i, (img_batch, mel_batch, frames, coords) in enumerate(gen):
    
    img_batch = torch.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2))).to(device)
    mel_batch = torch.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2))).to(device)
    
    with torch.no_grad():
        pred = model(mel_batch, img_batch)
    
    pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.

    print(time.time() - start)
    
    for p, f, c in zip(pred, frames, coords):
        y1, y2, x1, x2 = c
        p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))
        f[y1:y2, x1:x2] = p
        final_frames.append(f)
    print(time.time() - start)

Gen: 2.1823174953460693
0.7905793190002441
0.7980532646179199
0.8929417133331299
0.9000263214111328
0.9925119876861572
0.9997549057006836
1.092132329940796
1.0996448993682861
1.1917393207550049
1.1993117332458496
1.2915191650390625
1.2986290454864502
1.391007900238037
1.398115634918213
1.4901542663574219
1.49764084815979
1.5892763137817383
1.5968196392059326
1.688359022140503
1.6959052085876465
1.7882928848266602
1.7955622673034668
1.8869554996490479
1.893995761871338
1.9857127666473389
1.9928898811340332
2.0864803791046143
2.0936059951782227
2.1875345706939697
2.1946606636047363
2.2879467010498047
2.2951958179473877
2.3886430263519287
2.396155834197998
2.4887490272521973
2.49595046043396
2.5885658264160156
2.5957703590393066
2.688366651535034
2.6955318450927734
2.7879130840301514
2.795130491256714
2.8877625465393066
2.89536190032959
2.9873781204223633
2.994703531265259
3.0858800411224365
3.0931899547576904
3.185431480407715
3.192714214324951
3.285151481628418
3.292250871658325
3.38377

In [9]:
frame_h, frame_w = select_frames[0].shape[:-1]
out = cv2.VideoWriter('temp/result.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_w, frame_h))
for f in final_frames:
    out.write(f)
out.release()
# command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 {}'.format(args.audio, 'temp/result.avi', args.outfile)
# subprocess.call(command, shell=platform.system() != 'Windows')
# print("combine video & audio: ", time.time() - start)
# print(time.time() - start)

In [ ]:
import requests, time
start = time.time()
response = requests.post('http://54.179.106.170:5800/test', json = {"context": "haha\nhaha"})
print(time.time() - start)

In [ ]:
response.content

In [ ]:
from flask import Response
from __future__ import absolute_import, division, print_function
from flask import Flask
from flask import jsonify
from flask import request
from flask import send_from_directory

from flask_cors import CORS, cross_origin
from flask import Response

app = Flask(__name__)

CORS(app)
@app.route('/tts', methods=['POST'])
def post_tts():
    def generate():
        for frame in iter_all_rows():
            yield ','.join(row) + '\n'
    return Response(generate(), mimetype='text/plain')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5900,debug=False)

In [ ]:
from lipsyn import LipSyner

In [ ]:
model = LipSyner('checkpoints/wav2lip_gan.pth','data/1min.mp4')

In [ ]:
chunks = model.predict('data/input_audio2.wav',0)

In [ ]:
import time
start = time.time()
a = list(chunks)
print(time.time() - start)

In [ ]:
import time
start = time.time()
for c in chunks:
    print(time.time() - start)
    pass

In [ ]:
import pickle
pickle.dump(model.face_det_results,open('face_det_results_1min.pkl','wb'))

In [ ]:
import pickle
data = pickle.load(open('face_det_results_1min.pkl','rb'))

In [ ]:
len(data)

In [ ]:
i = 0
for chunk in chunks:
    
    byte_frames = chunk.split(b'endofframe')
    
    for frame in byte_frames[:-1]:
        i += 1
        with open('imgs/' + str(i) + '.jpeg','wb') as f:
            f.write(frame)
    if i > 30:
        break

In [ ]:
import pickle
pickle.dump(model.face_det_results,open('face_det_results.pkl','wb'))

In [ ]:
from lipsyn import LipSyner
from flask import Response
from flask import Flask
from flask import request
from flask_cors import CORS, cross_origin
import cv2

app = Flask(__name__)

model = LipSyner('checkpoints/wav2lip_gan.pth','data/input_vid.mp4','face_det_results.pkl')

CORS(app)
@app.route('/lip', methods=['POST'])
def post_lip():
    audio_path   = request.json['audio_path']
    start_index  = request.json['start_index']
    def generate_frame():
        frame_batch = []
        for frame in model.predict(audio_path,start_index):
            frame_batch.append(cv2.imencode('.jpg',frame)[1].tostring())
            if len(frame_batch) > 10:
                yield b'endofframe'.join(frame_batch)
                frame_batch = []
    
    return Response(generate_frame(),content_type = 'application/octet-stream')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5800,debug=False)

In [ ]:
import cv2
import time
start = time.time()
all_frames = []
cap = cv2.VideoCapture('data/v.mp4')
while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        break
    frame = cv2.imencode('.jpg',frame)[1].tostring()
    all_frames.append(frame)
print(time.time() - start)

In [ ]:
len(all_frames)

In [ ]:
6354 / 24

In [ ]:
from threading import Thread
import time

def updating_frame(threadname):
    for i in range(0,5):
        time.sleep(1)
        print(i)

# thread = Thread(target=updating_frame, args=("update", ))
# thread.start()

In [ ]:
thread = Thread(target=updating_frame, args=("update", ))
thread.start()

In [ ]:
def main_thead():
    thread = Thread(target=updating_frame, args=("update", ))
    thread.start()
    time.sleep(10)
    print('done main thread')
    
main_thead()

In [ ]:
thread.join()

In [ ]:
thread.start()